# SWITCH Input tables that deal with time
* timeseries
    - the dates picked in this table, impact the rest of the tables
* timepoints
* hydro_timepoints
* graph_timestamp_map
* loads
* variable_capacity_factors

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

import pandas as pd
import numpy as np
from datetime import datetime

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###
from pathlib import Path
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)

cwd = Path.cwd()

settings_path = (cwd / "settings_TD.yml")
settings = load_settings(settings_path)


In [2]:
# read in csv outputs from PG

#existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
#new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
#potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
#fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
#fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)
all_gen_variability = pd.read_csv('PG_output_csv/all_gen_variability_WECC.csv', index_col=0)

# timeseries

In [3]:
def timeseries(load_curves, max_weight, avg_weight, ts_duration_of_tp, ts_num_tps): #20.2778, 283.8889
    '''
    Create the timeseries table based on REAM Scenario 178
    Input:
        1) load_curves: created using PowerGenome make(final_load_curves(pg_engine, scenario_settings[][]))
        2) max_weight: the weight to apply to the days with highest values
        3) avg_weight: the weight to apply to the days with average value
        3) ts_duration_of_tp: how many hours should the timpoint last
        4) ts_num_tps: number of timpoints in the selected day
    Output columns:
        - TIMESERIES: format: yyyy_yyyy-mm-dd
        - ts_period: the period decade
        - ts_duration_of_tp: based on input value
        - ts_num_tps: based on input value. Should multiply to 24 with ts_duration_of_tp
        - ts_scale_to_period: use the max&avg_weights for the average and max days in a month
    '''
    hr_load_sum = pd.DataFrame(load_curves.sum(axis=1), columns=['sum_across_regions'])
    num_hrs = len(load_curves.index) # number of hours PG outputs data for in a year
    hr_interval = round(num_hrs / 8760)
    hr_int_list = list(range(1, int(24/hr_interval)+1))

    yr_dates = [d.strftime('%Y%m%d') for d in pd.date_range('20200101','20201231')]
    leap_yr = '20200229'
    yr_dates.remove(leap_yr)

    # create initial date list for 2020
    timestamp = list()
    for d in range(len(yr_dates)):
        for i in hr_int_list:
            date_hr = yr_dates[d]
            timestamp.append(date_hr)

    timeseries = [x[:4] +'_'+ x[:4] +'-'+ x[4:6] +'-'+ x[6:8] for x in timestamp]
    ts_period = [x[:4] for x in timestamp]
    timepoint_id = list(range(len(timestamp)))

    column_list = ['timeseries', 'ts_period']
    data = np.array([timeseries, ts_period]).T
    initial_df = pd.DataFrame(data, columns = column_list, index=hr_load_sum.index)
    initial_df = initial_df.join(hr_load_sum)
    
    month_hrs = [744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744]
    year_cumul = [744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760] # cumulative hours by month

    # split dataframe into months (grouped by day)
    M1_df = (initial_df.iloc[0:year_cumul[0], :]).groupby('timeseries').sum()
    M2_df = (initial_df.iloc[year_cumul[0]: year_cumul[1], :]).groupby('timeseries').sum()
    M3_df = (initial_df.iloc[year_cumul[1]: year_cumul[2], :]).groupby('timeseries').sum()
    M4_df = (initial_df.iloc[year_cumul[2]: year_cumul[3], :]).groupby('timeseries').sum()
    M5_df = (initial_df.iloc[year_cumul[3]: year_cumul[4], :]).groupby('timeseries').sum()
    M6_df = (initial_df.iloc[year_cumul[4]: year_cumul[5], :]).groupby('timeseries').sum()
    M7_df = (initial_df.iloc[year_cumul[5]: year_cumul[6], :]).groupby('timeseries').sum()
    M8_df = (initial_df.iloc[year_cumul[6]: year_cumul[7], :]).groupby('timeseries').sum()
    M9_df = (initial_df.iloc[year_cumul[7]: year_cumul[8], :]).groupby('timeseries').sum()
    M10_df = (initial_df.iloc[year_cumul[8]: year_cumul[9], :]).groupby('timeseries').sum()
    M11_df = (initial_df.iloc[year_cumul[9]: year_cumul[10], :]).groupby('timeseries').sum()
    M12_df = (initial_df.iloc[year_cumul[10]: year_cumul[11], :]).groupby('timeseries').sum()

    month_df = [M1_df, M2_df, M3_df, M4_df, M5_df, M6_df, M7_df, M8_df, M9_df, M10_df, M11_df, M12_df]
    
    # find mean and max for each month, add date to a dataframe  
    timeseries_df = pd.DataFrame(columns=['sum_across_regions', 'timeseries','close_to_mean'])
    for df in month_df:
        df['timeseries'] = df.index
        mean = df['sum_across_regions'].mean()
        df['close_to_mean'] = abs(df['sum_across_regions'] - mean)              
        df_mean = df.loc[df['close_to_mean']==df['close_to_mean'].min()]
        df_max = df.loc[df['sum_across_regions']==df['sum_across_regions'].max()]
        timeseries_df = timeseries_df.append(df_max)
        timeseries_df = timeseries_df.append(df_mean)
        timeseries_df['timeseries'] = timeseries_df.index
    
    
    # add in other columns
    timeseries_df['ts_period'] = '2020'
    timeseries_df['ts_duration_of_tp'] = ts_duration_of_tp # assuming 4 for now
    timeseries_df['ts_num_tps'] = ts_num_tps # assuming 6 for now
    timeseries_df = timeseries_df.reset_index(drop=True)
    timeseries_df = timeseries_df.drop(['sum_across_regions'], axis=1)
    
    timeseries_df['ts_scale_to_period'] = None
    
    for i in range(len(timeseries_df)):
        if i % 2 == 0:
            timeseries_df.loc[i, 'ts_scale_to_period'] = max_weight
    timeseries_df['ts_scale_to_period'].replace(to_replace=[None], value=avg_weight, inplace=True)
    
    # add in addtional years (just replace 2020 with new year)
    addtl_yrs = ['2030', '2040', '2050']
    addtl_df = pd.DataFrame(columns=timeseries_df.columns)
    for y in addtl_yrs:
        df = timeseries_df.copy()
        df['ts_period'] = y
        col1 = df['timeseries'].to_list()
        col1 = [y +'_'+ y + x[9:] for x in col1]
        df['timeseries'] = col1
        addtl_df = addtl_df.append(df)
    timeseries_df = timeseries_df.append(addtl_df)
    timeseries_df = timeseries_df[['timeseries', 'ts_period', 'ts_duration_of_tp', 'ts_num_tps', 'ts_scale_to_period']]
    return timeseries_df

In [4]:
timeseries_df = timeseries(load_curves, max_weight=20.2778, avg_weight=283.8889, ts_duration_of_tp=4, 
                          ts_num_tps=6)
# dates that should be used in the other tables
timeseries_dates = timeseries_df['timeseries'].to_list()
timeseries_df

,timeseries,ts_period,ts_duration_of_tp,ts_num_tps,ts_scale_to_period
0,2020_2020-01-17,2020,4,6,20.2778
1,2020_2020-01-26,2020,4,6,283.8889
2,2020_2020-02-28,2020,4,6,20.2778
3,2020_2020-02-24,2020,4,6,283.8889
4,2020_2020-03-01,2020,4,6,20.2778
...,...,...,...,...,...
19,2050_2050-10-25,2050,4,6,283.8889
20,2050_2050-11-28,2050,4,6,20.2778
21,2050_2050-11-05,2050,4,6,283.8889
22,2050_2050-12-19,2050,4,6,20.2778


# timepoints

In [5]:
def timepoints_table(timeseries_dates, timestamp_interval):
    '''
    Create the timepoints SWICH input file based on REAM Scenario 178
    Inputs:
        1) timeseries_dates: timeseries dates from the timeseries table
        2) timestamp_interval: based on ts_duration_of_tp and ts_num_tps from the timeseries table.  
                Should be between 0 and 24.
    Output columns:
        * timepoints_id: ID
        * timestamp: timeseries formatted yyymmddtt where tt is in the timestamp_inverval list
        * timeseries: the timesries date from the timeseries table
    '''
    timepoint_id = list(range(1, len(timeseries_dates)+1))
    timestamp = [x[:4] + x[10:12] + x[13:] for x in timeseries_dates]
    
    column_list = ['timepoint_id', 'timestamp', 'timeseries']
    timepoints_df = pd.DataFrame(columns = column_list)
    for i in timestamp_interval:
        timestamp_interval = [x+i for x in timestamp]
        df_data = np.array([timepoint_id, timestamp_interval, timeseries_dates]).T
        df = pd.DataFrame(df_data, columns=column_list)
        timepoints_df = timepoints_df.append(df)
    
    timepoints_df['timepoint_id'] = range(1, len(timepoints_df['timeseries'].to_list())+1)
    
    return timepoints_df
    

In [6]:
timestamp_interval = ['00', '04', '08', '12','16', '20'] # should align with ts_duration_of_tp and ts_num_tps
timepoints_df = timepoints_table(timeseries_dates, timestamp_interval)
timepoints_df

,timepoint_id,timestamp,timeseries
0,1,2020011700,2020_2020-01-17
1,2,2020012600,2020_2020-01-26
2,3,2020022800,2020_2020-02-28
3,4,2020022400,2020_2020-02-24
4,5,2020030100,2020_2020-03-01
...,...,...,...
91,572,2050102520,2050_2050-10-25
92,573,2050112820,2050_2050-11-28
93,574,2050110520,2050_2050-11-05
94,575,2050121920,2050_2050-12-19


In [7]:
# create lists and dictionary for later use
timepoints_timestamp = timepoints_df['timestamp'].to_list() # timestamp list
timepoints_tp_id = timepoints_df['timepoint_id'].to_list() # timepoint_id list
timepoints_dict = dict(zip(timepoints_timestamp, timepoints_tp_id)) # {timestamp: timepoint_id}

In [8]:
# check for duplicated days (april 30th was duplicating due it saying it was max and avg)
timepoints_df[timepoints_df.timestamp.duplicated(keep=False)]

,timepoint_id,timestamp,timeseries


# hydro_timepoints

In [9]:
def hydro_timepoints_table(timepoints_df):
    '''
    Create the hydro_timepoints table based on REAM Scenario 178
    Inputs:
        1) timepoints_df: the SWITCH timepoints table
    Output Columns
        * timepoint_id: from the timepoints table
        * tp_to_hts: format: yyyy_M#. Based on the timestamp date from the timepoints table 
    '''
    
    hydro_timepoints = timepoints_df
    hydro_timepoints = hydro_timepoints[['timepoint_id', 'timestamp']]
    convert_to_hts = {'01':'_M1', '02': '_M2', '03': '_M3', '04': '_M4', '05': '_M5', '06': '_M6', 
                     '07': '_M7', '08': '_M8', '09': '_M9', '10': '_M10', '11': '_M11', '12': '_M12'}
    def convert(tstamp):
        month = tstamp[4:6]
        year = tstamp[0:4]
        return year + convert_to_hts[month]
    
    hydro_timepoints['tp_to_hts'] = hydro_timepoints['timestamp'].apply(convert)
    hydro_timepoints.drop('timestamp', axis=1, inplace=True)
    
    return hydro_timepoints

In [10]:
hydro_timepoints_df = hydro_timepoints_table(timepoints_df)
hydro_timepoints_df

,timepoint_id,tp_to_hts
0,1,2020_M1
1,2,2020_M1
2,3,2020_M2
3,4,2020_M2
4,5,2020_M3
...,...,...
91,572,2050_M10
92,573,2050_M11
93,574,2050_M11
94,575,2050_M12


# graph timestamp map


In [11]:
def graph_timestamp_map_table(timeseries_df, timestamp_interval):
    '''
    Create the graph_timestamp_map table based on REAM Scenario 178
    Input:
        1) timeseries_df: the SWITCH timeseries table
        2) timestamp_interval:based on ts_duration_of_tp and ts_num_tps from the timeseries table.  
                Should be between 0 and 24.
    Output columns:
        * timestamp: dates based on the timeseries table
        * time_row: the period decade year based on the timestamp
        * time_column: format: yyyymmdd. Using 2012 because that is the year data is based on.
    '''
    
    timeseries_df_copy = timeseries_df.copy()
    timeseries_df_copy = timeseries_df_copy[['timeseries', 'ts_period']]
    # reformat timeseries for timestamp
    timeseries_df_copy['timestamp'] = timeseries_df_copy['timeseries'].apply(lambda x: x[5:9]+x[10:12]+x[13:])

    # add in intervals to the timestamp
    graph_timeseries_map = pd.DataFrame(columns=['timeseries','ts_period','timestamp'])
    for x in timestamp_interval:
        df = timeseries_df_copy[['timeseries', 'ts_period']]
        df['timestamp'] = timeseries_df_copy['timestamp'] + x
        graph_timeseries_map = graph_timeseries_map.append(df)

    # using 2012 for financial year
    graph_timeseries_map['time_column'] = graph_timeseries_map['timeseries'].apply(
        lambda x: str(2012)+x[10:12]+x[13:15])

    graph_timeseries_map = graph_timeseries_map.drop(['timeseries'],axis=1)
    graph_timeseries_map = graph_timeseries_map.rename(columns={'ts_period':'time_row'})
    graph_timeseries_map = graph_timeseries_map[['timestamp', 'time_row', 'time_column']]
    
    return graph_timeseries_map

In [12]:
timestamp_interval = ['00', '04', '08', '12', '16', '20'] # should align with ts_duration_of_tp and ts_num_tps

graph_timestamp_map = graph_timestamp_map_table(timeseries_df, timestamp_interval)
graph_timestamp_map

,timestamp,time_row,time_column
0,2020011700,2020,20120117
1,2020012600,2020,20120126
2,2020022800,2020,20120228
3,2020022400,2020,20120224
4,2020030100,2020,20120301
...,...,...,...
19,2050102520,2050,20121025
20,2050112820,2050,20121128
21,2050110520,2050,20121105
22,2050121920,2050,20121219


# loads

In [13]:
def loads_table(load_curves, timepoints_timestamp, timepoints_dict, period_list):
    '''
    Inputs:
        load_curves: from powergenome
        timepoints_timestamp: the timestamps in timepoints
        timepoints_dict: to go from timestamp to timepoint
        period_list: the decade list
    Output columns
        * load_zone: the IPM regions
        * timepoint: from timepoints
        * zone_demand_mw: based on load_curves
    Output df
        loads: the 'final' table
        loads_with_hour_year: include hour year so it is easier to do variable_capacity_factors
    '''
    
    loads_initial = pd.DataFrame(columns=['year_hour', 'LOAD_ZONE', 'zone_demand_mw'])
    hours = load_curves.index.to_list()
    cols = load_curves.columns.to_list()
    
    # add load zone for each hour of the year, adding in the load_curve values for each hour
    for c in cols:
        df = pd.DataFrame()
        df['year_hour'] = hours
        df['LOAD_ZONE'] = c
        df['zone_demand_mw'] = load_curves[c].to_list()
        loads_initial = loads_initial.append(df)

    # convert timepoints to date of the year
    start = pd.to_datetime('2021-01-01 0:00')  # use 2021 due to 2020 being a leap year
    loads_initial['date'] = loads_initial['year_hour'].apply(lambda x: start + pd.to_timedelta(x, unit='H'))
    # reformat to timestamp format
    loads_initial['reformat'] = loads_initial['date'].apply(lambda x: x.strftime("%Y%m%d%H"))
    loads_initial['reformat'] = loads_initial['reformat'].astype(str)
    # create timestamp
    date_list = loads_initial['reformat'].to_list()
    
    # loop through for each period
    df_list = list()
    for p in period_list:
        df = loads_initial.copy()
        updated_dates = [p + x[4:] for x in date_list]
        df['timestamp'] = updated_dates
        df_list.append(df)
    loads = pd.concat(df_list)
    
    # filter to correct timestamps for timepoints
    loads = loads[loads['timestamp'].isin(timepoints_timestamp)]
    loads['TIMEPOINT'] = loads['timestamp'].apply(lambda x: timepoints_dict[x])
    loads_with_year_hour = loads[['timestamp','TIMEPOINT', 'year_hour']]
    loads = loads[['LOAD_ZONE', 'TIMEPOINT', 'zone_demand_mw']]
    
    return loads, loads_with_year_hour

In [14]:
period_list = ['2020', '2030', '2040','2050']
loads, loads_with_year_hour = loads_table(load_curves, timepoints_timestamp, timepoints_dict, period_list)
loads

,LOAD_ZONE,TIMEPOINT,zone_demand_mw
383,WECC_AZ,1,10466
387,WECC_AZ,97,8741
391,WECC_AZ,193,10058
395,WECC_AZ,289,10521
399,WECC_AZ,385,9450
...,...,...,...
8451,WEC_SDGE,191,2076
8455,WEC_SDGE,287,2069
8459,WEC_SDGE,383,2599
8463,WEC_SDGE,479,2446


In [15]:
# for fuel_cost and regional_fuel_market issue
dummy_df = pd.DataFrame({'TIMEPOINT':timepoints_tp_id})
dummy_df.insert(0,'LOAD_ZONE','loadzone')
dummy_df.insert(2,'zone_demand_mw',0)

In [16]:
loads = loads.append(dummy_df)
loads

,LOAD_ZONE,TIMEPOINT,zone_demand_mw
383,WECC_AZ,1,10466
387,WECC_AZ,97,8741
391,WECC_AZ,193,10058
395,WECC_AZ,289,10521
399,WECC_AZ,385,9450
...,...,...,...
571,loadzone,572,0
572,loadzone,573,0
573,loadzone,574,0
574,loadzone,575,0


In [17]:
loads_with_year_hour

,timestamp,TIMEPOINT,year_hour
383,2020011700,1,384
387,2020011704,97,388
391,2020011708,193,392
395,2020011712,289,396
399,2020011716,385,400
...,...,...,...
8451,2050121904,191,8452
8455,2050121908,287,8456
8459,2050121912,383,8460
8463,2050121916,479,8464


In [18]:
year_hour = loads_with_year_hour['year_hour'].to_list()

# variable_capacity_factors

In [19]:
def variable_capacity_factors_table(all_gen_variability, year_hour, timepoints_dict, all_gen):
    '''
    Inputs
        all_gen_variability: from powergenome
        year_hour: the hour of the year that has a timepoint (based on loads)
        timepoints_dict: convert timestamp to timepoint
        all_gen: from powergenome
    Output:
        GENERATION_PROJECT: based on all_gen index
            the plants here should only be the ones with gen_is_variable =True
        timepoint: based on timepoints
        gen_max_capacity_factor: based on all_gen_variability
    '''
    
    v_capacity_factors = all_gen_variability.copy().transpose()
    v_capacity_factors['GENERATION_PROJECT'] = v_capacity_factors.index
    v_c_f = v_capacity_factors.melt(id_vars='GENERATION_PROJECT', var_name='year_hour', 
                                    value_name='gen_max_capacity_factor')
    # reduce variability to just the hours of the year that have a timepoint
    v_c_f = v_c_f[v_c_f['year_hour'].isin(year_hour)]
    
    mod_vcf = v_c_f.copy()
    # get the dates from hour of the year
    start = pd.to_datetime('2021-01-01 0:00') # 2020 is a leap year
    mod_vcf['date'] = mod_vcf['year_hour'].apply(lambda x: start + pd.to_timedelta(x, unit='H'))
    mod_vcf['reformat'] = mod_vcf['date'].apply(lambda x: x.strftime("%Y%m%d%H"))
    mod_vcf['reformat'] = mod_vcf['reformat'].astype(str)
    date_list = mod_vcf['reformat'].to_list()
    # change 2021 to correct period year/decade
    # to get the timestamp
    updated_dates20 = ['2020' + x[4:] for x in date_list]
    updated_dates30 = ['2030' + x[4:] for x in date_list]
    updated_dates40 = ['2040' + x[4:] for x in date_list]
    updated_dates50 = ['2050' + x[4:] for x in date_list]
    mod_vcf1 = mod_vcf.copy()
    mod_vcf2 = mod_vcf.copy()
    mod_vcf3 = mod_vcf.copy()
    mod_vcf4 = mod_vcf.copy()
    mod_vcf1['timestamp'] = updated_dates20
    mod_vcf2['timestamp'] = updated_dates30
    mod_vcf3['timestamp'] = updated_dates40
    mod_vcf4['timestamp'] = updated_dates50
    # go from timestamp to timepoint
    mod_vcf1['timepoint'] = mod_vcf1['timestamp'].apply(lambda x: timepoints_dict[x])
    mod_vcf2['timepoint'] = mod_vcf2['timestamp'].apply(lambda x: timepoints_dict[x])
    mod_vcf3['timepoint'] = mod_vcf3['timestamp'].apply(lambda x: timepoints_dict[x])
    mod_vcf4['timepoint'] = mod_vcf4['timestamp'].apply(lambda x: timepoints_dict[x])
    # get final columns
    mod_vcf1.drop(['year_hour','date','reformat','timestamp'], axis=1, inplace=True)
    mod_vcf2.drop(['year_hour','date','reformat','timestamp'], axis=1, inplace=True)
    mod_vcf3.drop(['year_hour','date','reformat','timestamp'], axis=1, inplace=True)
    mod_vcf4.drop(['year_hour','date','reformat','timestamp'], axis=1, inplace=True)
    # bring all decades together
    var_cap_fac = pd.concat([mod_vcf1, mod_vcf2, mod_vcf3, mod_vcf4], ignore_index=True)
    
    # only get all_gen plants that are wind or solar
    technology = all_gen['technology'].to_list()
    def Filter(list1, list2):
        return [n for n in list1 if
                 any(m in n for m in list2)]
    wind_solar = set(Filter(technology, ['Wind', 'Solar']))
    all_gen.loc[all_gen['technology'].isin(wind_solar), 'gen_is_variable'] = True
    all_gen = all_gen[all_gen['gen_is_variable']==True]
    
    # get the correct GENERATION_PROJECT instead of region_resource_cluster from variability table
    all_gen = all_gen.copy()
    all_gen['region_resource_cluster'] = all_gen["region"]+ "_"+ all_gen["Resource"
                                                            ] + "_" + all_gen["cluster"].astype(str)
    all_gen['gen_id'] = all_gen.index
    all_gen_convert = dict(zip(all_gen['region_resource_cluster'].to_list(), all_gen['gen_id'].to_list()))
    
    reg_res_cl = all_gen['region_resource_cluster'].to_list()
    var_cap_fac = var_cap_fac[var_cap_fac['GENERATION_PROJECT'].isin(reg_res_cl)]
    
    var_cap_fac['GENERATION_PROJECT'] = var_cap_fac['GENERATION_PROJECT'].apply(lambda x: all_gen_convert[x])
    # filter to final columns
    var_cap_fac = var_cap_fac[['GENERATION_PROJECT', 'timepoint', 'gen_max_capacity_factor']]
    var_cap_fac['GENERATION_PROJECT'] = var_cap_fac['GENERATION_PROJECT'] +1  # switch error - can't be 0?
#     vcf['GENERATION_PROJECT'] = vcf['GENERATION_PROJECT'].astype(str)
    
    return var_cap_fac

In [20]:
vcf = variable_capacity_factors_table(all_gen_variability, year_hour, timepoints_dict, all_gen)

In [21]:
vcf

,GENERATION_PROJECT,timepoint,gen_max_capacity_factor
154,155,1,0.2686
155,156,1,0.2686
156,157,1,0.2686
157,158,1,0.2686
158,159,1,0.2686
...,...,...,...
2553323,4349,575,0.0000
2553324,4350,575,0.0000
2553325,4351,575,0.0000
2553326,4352,575,0.0000


In [22]:
timeseries_df.to_csv(r'SWITCH_Inputs\timeseries.csv', index = False)
timepoints_df.to_csv(r'SWITCH_Inputs\timepoints.csv', index = False)
hydro_timepoints_df.to_csv(r'SWITCH_Inputs\hydro_timepoints.csv', index = False)
graph_timestamp_map.to_csv(r'SWITCH_Inputs\graph_timestamp_map.csv', index = False)
loads.to_csv(r'SWITCH_Inputs\loads.csv', index = False)
vcf.to_csv(r'SWITCH_Inputs\variable_capacity_factors.csv', index = False)